In [5]:
import pymatgen
from pymatgen.core.structure import Structure, Lattice

import numpy as np
import os

ImportError: cannot import name 'Structure' from 'pymatgen' (unknown location)

In [3]:
data_dir = '/data/qgding/intel/data'
cif_dir = os.path.join(data_dir, 'mp_cif')

In [4]:
def get_crystal_string(structure):
    # Randomly translate within the unit cell
    structure.translate_sites(
        indices=range(len(structure.sites)), vector=np.random.uniform(size=(3,))
    )

    lengths = structure.lattice.parameters[:3]
    angles = structure.lattice.parameters[3:]
    atom_ids = structure.species
    frac_coords = structure.frac_coords

    crystal_str = \
        " ".join(["{0:.1f}".format(x) for x in lengths]) + "\n" + \
        " ".join([str(int(x)) for x in angles]) + "\n" + \
        "\n".join([
            str(t) + "\n" + " ".join([
                "{0:.2f}".format(x) for x in c
            ]) for t,c in zip(atom_ids, frac_coords)
        ])

    return crystal_str

def parse_fn(gen_str):
    lines = [x for x in gen_str.split("\n") if len(x) > 0]
    lengths = [float(x) for x in lines[0].split(" ")]
    angles = [float(x) for x in lines[1].split(" ")]
    species = [x for x in lines[2::2]]
    coords = [[float(y) for y in x.split(" ")] for x in lines[3::2]]
    
    structure = Structure(
        lattice=Lattice.from_parameters(
            *(lengths + angles)),
        species=species,
        coords=coords, 
        coords_are_cartesian=False,
    )
    
    return structure.to(fmt="cif")

In [8]:
for cif in os.listdir(cif_dir):
    if cif.endswith('.cif'):
        structure = Structure.from_file(os.path.join(cif_dir, cif))
        crystal_str = get_crystal_string(structure)
        print(cif)
        print(structure)
        print(crystal_str)
        break

mp-1195070.cif
Full Formula (Zr16 Cu4 Sb28)
Reduced Formula: Zr4CuSb7
abc   :  11.290148  11.290148   8.772933
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (48)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Zr    0.972223  0.997673  0.464865
  1  Zr    0.472223  0.497673  0.464865
  2  Zr    0.217964  0.743414  0.456894
  3  Zr    0.726482  0.251932  0.456894
  4  Zr    0.717964  0.751932  0.456894
  5  Zr    0.226482  0.243414  0.456894
  6  Zr    0.472223  0.997673  0.448435
  7  Zr    0.972223  0.497673  0.448435
  8  Zr    0.477149  0.74647   0.983977
  9  Zr    0.467297  0.248876  0.983977
 10  Zr    0.977149  0.748876  0.983977
 11  Zr    0.967297  0.24647   0.983977
 12  Zr    0.223426  0.502599  0.983977
 13  Zr    0.72102   0.492747  0.983977
 14  Zr    0.22102   0.002599  0.983977
 15  Zr    0.723426  0.992747  0.983977
 16  Cu    0.346342  0.871792  0.715676
 17  Cu    0.598104  0.123554  0.715

In [9]:
mp_id = 'mp-1000'
cif = os.path.join(cif_dir, mp_id + '.cif')
structure = Structure.from_file(cif)
crystal_str = get_crystal_string(structure)
print(mp_id)
print(structure)
print(crystal_str)

mp-1000
Full Formula (Ba1 Te1)
Reduced Formula: BaTe
abc   :   5.013273   5.013274   5.013273
angles:  60.000003  59.999999  60.000004
pbc   :       True       True       True
Sites (2)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Ba    0.331349  0.494842  0.091289
  1  Te    0.831349  0.994842  0.591289
5.0 5.0 5.0
60 59 60
Ba
0.33 0.49 0.09
Te
0.83 0.99 0.59


In [10]:
def cartesian_to_fractional(lattice_vectors, cartesian_coords):
    """
    Converts Cartesian coordinates to fractional coordinates.
    
    Parameters:
    lattice_vectors (np.ndarray): A 3x3 numpy array where each row is a lattice vector.
    cartesian_coords (np.ndarray): A 1D numpy array representing the Cartesian coordinates.
    
    Returns:
    np.ndarray: A 1D numpy array representing the fractional coordinates.
    """
    M = np.transpose(lattice_vectors)
    f = np.linalg.inv(M).dot(cartesian_coords)
    return f

In [11]:
# Define the lattice vectors
a = np.array([1, 0, 0])
b = np.array([0, 2, 0])
c = np.array([0, 0, 3])
lattice_vectors = np.array([a, b, c])

# Define the Cartesian coordinates of the atom
r = np.array([1, 2, 3])

# Call the function
fractional_coords = cartesian_to_fractional(lattice_vectors, r)

print(fractional_coords)

[1. 1. 1.]


In [14]:
lattice_vectors = [
            [
              0,
              3.544879e-10,
              3.544879e-10
            ],
            [
              3.544879e-10,
              0,
              3.544879e-10
            ],
            [
              3.544879e-10,
              3.544879e-10,
              0
            ]
          ]

cartesian_coords = [
            [
              3.5448789999999993e-10,
              3.5448789999999993e-10,
              3.5448789999999993e-10
            ],
            [
              0,
              2.0697386543421307e-26,
              2.0697386543421307e-26
            ]
          ]

fractional_coords = cartesian_to_fractional(lattice_vectors, cartesian_coords[0])
print(fractional_coords)
fractional_coords = cartesian_to_fractional(lattice_vectors, cartesian_coords[1])
print(fractional_coords)

[0.5 0.5 0.5]
[ 5.83867222e-17  2.43601357e-33 -2.43601357e-33]
